Will create data for days 0-750 then fit a forecaster on it to predict logReturns and then export the model and attempt to runin a sim without retraining.

In [1]:
import glob
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import display
from skforecast.plot import set_dark_theme
from tqdm import tqdm

import sklearn
import skforecast
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.model_selection import (
    TimeSeriesFold,
    OneStepAheadFold,
    backtesting_forecaster,
    bayesian_search_forecaster,
    backtesting_forecaster_multiseries,
    bayesian_search_forecaster_multiseries
)
from skforecast.preprocessing import RollingFeatures, series_long_to_dict, exog_long_to_dict
from skforecast.exceptions import OneStepAheadValidationWarning

import warnings

In [2]:
prices1000Days = np.loadtxt("../../sourceCode/1000Prices.txt").T

from strategies.ms_forecasting.main import createGreeksManager
from strategies.ms_forecasting.main import GreeksManager
from strategies.ms_forecasting.main import PRICE_LAGS
from strategies.ms_forecasting.main import MOMENTUM_WINDOWS
from strategies.ms_forecasting.main import VOL_WINDOWS

preTrainingDays = max(PRICE_LAGS + MOMENTUM_WINDOWS + VOL_WINDOWS)

trainingDataWindowSize = 750 - preTrainingDays # 20 days of data needed for all greeks to be void of NaN's
gm: GreeksManager = createGreeksManager(prices1000Days[:, :751], trainingDataWindowSize) # giving 751 days but the 751'st is removed when we get the history

exogIndex = pd.RangeIndex(preTrainingDays, 750)
exogDict = gm.getGreeksHistoryDict(exogIndex)

print("Inst 0 days [742, 750]:\n", prices1000Days[0, 742:750])
display("inst 0 head:", exogDict["inst_0"].tail(1))
print(f"each inst-greek has {len(exogDict["inst_0"]["greek_lag_1"])} days")

Inst 0 days [742, 750]:
 [36.81 36.75 36.53 36.49 36.3  36.58 36.43 36.24]


'inst 0 head:'

,greek_lag_1,greek_lag_2,greek_lag_3,greek_lag_4,greek_lag_5,greek_lag_6,greek_lag_7,greek_volatility_5,greek_volatility_10,greek_volatility_20,greek_momentum_3,greek_momentum_7,greek_momentum_14,greek_price
749,36.43,36.58,36.3,36.49,36.53,36.75,36.81,0.005455,0.004396,0.00372,-0.001654,-0.015606,-0.028562,36.24


each inst-greek has 730 days


In [3]:
logReturnsPricesWindow = prices1000Days[:, preTrainingDays:]
logReturnsNp = np.log(logReturnsPricesWindow[:, 1:] / logReturnsPricesWindow[:, :-1])

logReturnsIndex = pd.RangeIndex(preTrainingDays, 999)
logReturnsSeries = pd.DataFrame(logReturnsNp.T,
                          index = logReturnsIndex,
                          columns = [f"inst_{i}" for i in range(logReturnsNp.shape[0])])

logReturnsTrainingSeries = logReturnsSeries.iloc[:trainingDataWindowSize]

In [4]:
logReturnsForecaster = ForecasterRecursiveMultiSeries(
    regressor           = HistGradientBoostingRegressor(random_state=8523,
                                                        learning_rate=0.05),
    transformer_series  = None,
    transformer_exog    = StandardScaler(),
    lags                = 7,
    window_features     = RollingFeatures(
                                stats           = ['min', 'max', 'ewm'],
                                window_sizes    = 7,
                            ),
)

logReturnsForecaster.fit(
    series  = logReturnsTrainingSeries,
    exog    = exogDict
)

logReturnsForecaster.is_fitted

True

In [5]:
display(logReturnsTrainingSeries.tail(1))
display(exogDict["inst_0"].tail(1))

,inst_0,inst_1,inst_2,inst_3,inst_4,inst_5,inst_6,inst_7,inst_8,inst_9,...,inst_40,inst_41,inst_42,inst_43,inst_44,inst_45,inst_46,inst_47,inst_48,inst_49
749,-0.001104,0.0,-0.009485,-0.010395,-0.003002,0.002784,0.002259,-0.008953,-0.007064,0.009198,...,0.0028,0.010673,-0.018626,-0.01809,-0.00267,0.001937,-0.006005,0.010743,-0.002986,-0.003171


,greek_lag_1,greek_lag_2,greek_lag_3,greek_lag_4,greek_lag_5,greek_lag_6,greek_lag_7,greek_volatility_5,greek_volatility_10,greek_volatility_20,greek_momentum_3,greek_momentum_7,greek_momentum_14,greek_price
749,36.43,36.58,36.3,36.49,36.53,36.75,36.81,0.005455,0.004396,0.00372,-0.001654,-0.015606,-0.028562,36.24


# Simulate updating the greeks daily and making a one step prediction:

In [7]:
predictions = []

for day in range(750, 1000):
    gm.updateGreeks(prices1000Days[:, day])

    currentDayIndex = pd.RangeIndex(start=day, stop=day + 1)
    predictionExogs = gm.getGreeksDict(currentDayIndex)
    logReturnsPredictionSeries = logReturnsSeries.iloc[:day].tail(7)
    #
    # print("log returns prediction series:")
    # display(logReturnsPredictionSeries)
    # print("Exogs inst_0:")
    # display(predictionExogs["inst_0"])
    # prediction = logReturnsForecaster.predict(
    #     last_window = logReturnsPredictionSeries,
    #     exog        = predictionExogs,
    #     steps       = 1,
    #     levels      = list(logReturnsSeries.columns),
    # )

    # predictions.append(prediction["pred"].values)

log returns prediction series:


,inst_0,inst_1,inst_2,inst_3,inst_4,inst_5,inst_6,inst_7,inst_8,inst_9,...,inst_40,inst_41,inst_42,inst_43,inst_44,inst_45,inst_46,inst_47,inst_48,inst_49
763,-0.004172,0.000405,-0.004673,0.000333,-0.007897,0.008472,-0.007870,0.014614,-0.001626,0.000180,...,-0.001204,-0.008116,0.008467,-0.016205,-0.012490,0.009056,0.003718,0.005734,-0.013466,0.010903
764,-0.000836,0.003840,-0.020173,0.002326,0.002809,-0.001407,-0.009070,0.004406,0.014269,0.015319,...,0.002272,0.004170,0.005286,-0.007569,-0.011376,-0.009730,-0.002090,-0.002545,0.007845,0.007801
765,-0.001116,-0.001615,0.010760,-0.008667,-0.005627,-0.010260,0.017500,0.005636,-0.014269,0.002295,...,0.003864,-0.011510,0.000479,0.009138,0.002962,-0.004393,0.009370,0.001909,-0.005137,0.013654
766,-0.003637,0.006646,-0.009754,-0.002682,0.004606,-0.000711,-0.010689,-0.003336,-0.003803,0.014532,...,-0.002797,-0.012709,0.022031,-0.022202,-0.002962,0.006077,0.008941,0.000000,-0.002191,-0.005024
767,-0.006185,0.004806,0.001507,-0.018293,-0.009491,-0.010374,0.001131,-0.001254,0.009212,0.004682,...,-0.003072,0.007222,0.010257,0.012430,-0.000565,-0.001684,0.018654,0.001271,0.001934,-0.003265
768,-0.001129,0.003589,-0.030576,-0.002053,0.009746,-0.002160,0.009559,-0.018787,0.013128,0.024271,...,-0.003350,-0.012351,-0.007449,0.014778,0.004934,0.006386,0.013241,0.000000,-0.006202,0.001188
769,-0.007083,0.004369,0.017695,-0.011023,0.000765,-0.002165,-0.027803,0.007642,0.028858,-0.004061,...,-0.000806,0.005769,-0.023163,0.004360,0.010909,-0.001676,0.001326,-0.002543,0.003880,0.000594


Exogs inst_0:


,greek_lag_1,greek_lag_2,greek_lag_3,greek_lag_4,greek_lag_5,greek_lag_6,greek_lag_7,greek_volatility_5,greek_volatility_10,greek_volatility_20,greek_momentum_3,greek_momentum_7,greek_momentum_14,greek_price
750,36.2,36.2,36.24,36.43,36.58,36.3,36.49,0.002431,0.003996,0.00364,-0.001104,-0.007979,-0.018881,36.2


log returns prediction series:


...

,greek_lag_1,greek_lag_2,greek_lag_3,greek_lag_4,greek_lag_5,greek_lag_6,greek_lag_7,greek_volatility_5,greek_volatility_10,greek_volatility_20,greek_momentum_3,greek_momentum_7,greek_momentum_14,greek_price
998,32.59,32.88,33.0,32.73,32.53,32.49,32.47,0.007016,0.005093,0.005012,-0.013116,0.003075,0.014847,32.57


log returns prediction series:


,inst_0,inst_1,inst_2,inst_3,inst_4,inst_5,inst_6,inst_7,inst_8,inst_9,...,inst_40,inst_41,inst_42,inst_43,inst_44,inst_45,inst_46,inst_47,inst_48,inst_49
992,0.001230,0.015498,-0.002046,-0.002596,-0.003988,0.000000,-0.023208,0.001327,0.023888,0.007895,...,-0.001032,-0.014389,-0.005238,-0.003969,-0.006583,0.002922,-0.007702,0.000000,0.003909,0.003236
993,0.006129,-0.004347,0.006330,-0.004342,0.001879,0.003219,0.022031,-0.003988,-0.018674,-0.000576,...,0.001548,-0.001706,-0.002103,-0.016035,0.000574,0.006543,-0.018403,0.001235,-0.001657,-0.001437
994,0.008215,0.004347,0.010931,-0.002905,-0.011564,-0.001608,0.009962,0.010379,0.008414,-0.008286,...,-0.002582,0.010196,-0.007042,0.020364,-0.004458,-0.001088,0.001930,-0.008680,-0.014193,-0.011569
995,-0.003643,0.001182,-0.002419,-0.000582,-0.012660,-0.000402,-0.012320,0.013312,-0.008414,0.033302,...,0.001421,0.010093,-0.006025,0.007888,-0.001009,0.001450,0.007829,0.002488,0.000120,-0.003278
996,-0.008859,0.004519,0.006638,-0.018511,-0.000481,0.006421,-0.003548,-0.011337,0.012595,-0.017940,...,0.002964,0.011372,-0.014682,-0.002145,-0.001588,-0.005812,-0.015421,0.007426,0.001320,0.001458
997,-0.000614,-0.005504,0.005996,0.015303,-0.001444,0.000400,-0.011320,0.003065,0.011487,-0.002862,...,-0.005549,0.003579,0.000721,-0.003226,-0.003909,-0.005846,-0.001047,-0.013656,-0.000600,0.003273
998,-0.002152,-0.000789,-0.017692,-0.007328,-0.003378,-0.010450,-0.010843,0.022905,-0.007004,0.004956,...,0.001164,-0.003303,0.009686,0.012844,-0.005528,-0.007356,0.008045,-0.003130,0.004550,0.013346


Exogs inst_0:


,greek_lag_1,greek_lag_2,greek_lag_3,greek_lag_4,greek_lag_5,greek_lag_6,greek_lag_7,greek_volatility_5,greek_volatility_10,greek_volatility_20,greek_momentum_3,greek_momentum_7,greek_momentum_14,greek_price
999,32.57,32.59,32.88,33.0,32.73,32.53,32.49,0.006211,0.005214,0.005018,-0.011624,0.000308,0.016442,32.5
